In [1]:
import glob
import json
import pandas as pd
import numpy as np
import datetime
from math import inf

Features to extract :
- Average cost of character (# / @) on he whole dataset
- Average use of characters (@ / #) for each user (N_usages / N_tweets)
- Frequency of tweets published (N_tweets / Delta_t dataset)
- Frequency of follow actions (Delta_friends / Delta_t, if positive)

In [16]:
min_timestamp = inf
max_timestamp = 0
cost_mention = []
cost_hashtag = []
N_Tweets = 0

In [17]:
data_folder = "data/Woldcup2008/Tweet Worldcup/raw"
files = glob.glob(f"{data_folder}/*.json")
print(f"Found {len(files)} files")

Found 2285 files


In [18]:
users_dict = {}

for i, file in enumerate(files[:500]) :

    if i % 100 == 0 and i > 0:
        print(f"Opening file {i}/{len(files)}")

    tweets = []
    with open(file, 'r', encoding='utf-8') as f:
        for line in f: 
            tweet = json.loads(line)
            tweets.append(tweet)

    for tweet in tweets:

        # User data

        user_id = tweet['user']['id']

        # Create the user if not already in the dict
        if user_id not in users_dict.keys():
            users_dict[user_id] = {
                'n_tweets' : 0,
                'max_friends' : 0,
                'min_friends' : inf,
                'max_followers' : 0,
                'n_mention' : 0,
                'n_hashtag' : 0,
                'n_urls' : 0,
                'first_active' : inf,
                'last_active' : 0,
            }

        # Add the tweet to the user count
        users_dict[user_id]['n_tweets'] += 1
        users_dict[user_id]['n_mention'] += len(tweet['entities']['user_mentions'])
        users_dict[user_id]['n_hashtag'] += len(tweet['entities']['hashtags'])
        users_dict[user_id]['n_urls'] += len(tweet['entities']['urls'])


        # Update min and max friends if necessary
        n_friends = tweet['user']['friends_count']
        if  n_friends > users_dict[user_id]['max_friends']:
            users_dict[user_id]['max_friends'] = n_friends

        if n_friends < users_dict[user_id]['min_friends']:
            users_dict[user_id]['min_friends'] = n_friends

        # Update max followers if necessary
        followers_count = tweet['user']['followers_count']
        if followers_count > users_dict[user_id]['max_followers']:
            users_dict[user_id]['max_followers'] = followers_count

        # Update first and last active time if necessary
        tweet_time = tweet['current_time']
        if tweet_time < users_dict[user_id]['first_active']:
            users_dict[user_id]['first_active'] = tweet_time
        if tweet_time > users_dict[user_id]['last_active']:
            users_dict[user_id]['last_active'] = tweet_time


        # Global data

        
        # Get the cost of each #
        for hashtag in tweet['entities']['hashtags']:
            cost_hashtag.append(len(hashtag['text']) + 1)

        # Get the cost of each @
        for mention in tweet['entities']['user_mentions']:
            cost_mention.append(len(mention['screen_name']) + 1)

        # Update min and max timestamp if necessary
        if tweet['current_time'] < min_timestamp:
            min_timestamp = tweet['current_time']
        elif tweet['current_time'] > max_timestamp:
            max_timestamp = tweet['current_time'] 

Opening file 100/2285
Opening file 200/2285
Opening file 300/2285
Opening file 400/2285


Execution time estimation :
- 10 files : 5"
- 100 files : 53"
- 200 files : 1'58"
- 500 files : 5'32"

In [19]:
# Compute the delta time (in microseconds)
delta_t = max_timestamp - min_timestamp
delta_t_h = delta_t / 3600000

In [20]:
print(f"Total number of # : {len(cost_hashtag)}")
print(f"Average cost of # : {np.mean(cost_hashtag):.2f} chars")
print(f"Total number of Mentions (@) : {len(cost_mention)}")
print(f"Average cost of Mentions (@) # : {np.mean(cost_mention):.2f} chars")
print(f"First tweet : {datetime.datetime.fromtimestamp(min_timestamp / 1000)}")
print(f"Last tweet : {datetime.datetime.fromtimestamp(max_timestamp / 1000)}")

Total number of # : 1638497
Average cost of # : 8.71 chars
Total number of Mentions (@) : 895104
Average cost of Mentions (@) # : 11.40 chars
First tweet : 2018-06-14 04:14:25.098000
Last tweet : 2018-06-16 13:43:00.391000


Create dataframe

In [21]:
users = pd.DataFrame.from_dict(users_dict, orient='index')
users = users.reset_index()      
users = users.rename(columns={'index': 'user_id'}) 
users.head()

,user_id,n_tweets,max_friends,min_friends,max_followers,n_mention,n_hashtag,n_urls,first_active,last_active
0,39084553,1,826,826,556,1,2,0,1528942465098,1528942465098
1,1564678657,1,686,686,918,0,0,1,1528942465150,1528942465150
2,967244973824339968,1,44,44,34,1,0,1,1528942465165,1528942465165
3,4872447178,2,3189,3189,2807,2,3,0,1528942465499,1528948008587
4,35628099,1,407,407,48,2,2,0,1528942466051,1528942466051


In [22]:
# Compute average use of # and @
users['average_use_mention'] = users['n_mention'] / users['n_tweets']
users['average_use_hashtag'] = users['n_hashtag'] / users['n_tweets']

In [23]:
# Compute frequency of actions (by hour) on the active delta time for users with +1 tweet / on the global delta time for others
i_mult_tweets = users['n_tweets'] > 1

users.loc[i_mult_tweets, 'f_tweets'] = users.loc[i_mult_tweets, 'n_tweets'] / ((users.loc[i_mult_tweets, 'last_active'] - users.loc[i_mult_tweets, 'first_active']) / 3600000)
users.loc[i_mult_tweets, 'f_friends'] = (users.loc[i_mult_tweets, 'max_friends'] - users.loc[i_mult_tweets, 'min_friends']) / ((users.loc[i_mult_tweets, 'last_active'] - users.loc[i_mult_tweets, 'first_active']) / 3600000)

users.loc[~ i_mult_tweets, 'f_tweets'] = users.loc[~ i_mult_tweets, 'n_tweets'] / delta_t_h
users.loc[~ i_mult_tweets, 'f_friends'] = (users.loc[~ i_mult_tweets, 'max_friends'] - users.loc[~ i_mult_tweets, 'min_friends']) / delta_t_h

In [24]:
users.head()

,user_id,n_tweets,max_friends,min_friends,max_followers,n_mention,n_hashtag,n_urls,first_active,last_active,average_use_mention,average_use_hashtag,f_tweets,f_friends
0,39084553,1,826,826,556,1,2,0,1528942465098,1528942465098,1.0,2.0,0.017398,0.0
1,1564678657,1,686,686,918,0,0,1,1528942465150,1528942465150,0.0,0.0,0.017398,0.0
2,967244973824339968,1,44,44,34,1,0,1,1528942465165,1528942465165,1.0,0.0,0.017398,0.0
3,4872447178,2,3189,3189,2807,2,3,0,1528942465499,1528948008587,1.0,1.5,1.298915,0.0
4,35628099,1,407,407,48,2,2,0,1528942466051,1528942466051,2.0,2.0,0.017398,0.0


Save Dataframe

In [25]:
users.to_csv('data/users.csv', index=False)

Load Dataframe

In [ ]:
# df = pd.read_csv('data/users.csv')
# df.head()

,user_id,n_tweets,max_friends,min_friends,max_followers,n_mention,n_hashtag,n_urls,first_active,last_active,average_use_mention,average_use_hashtag,f_tweets,f_friends
0,39084553,1,826,826,556,1,2,0,1528942465098,1528942465098,1.0,2.0,0.017398,0.0
1,1564678657,1,686,686,918,0,0,1,1528942465150,1528942465150,0.0,0.0,0.017398,0.0
2,967244973824339968,1,44,44,34,1,0,1,1528942465165,1528942465165,1.0,0.0,0.017398,0.0
3,4872447178,2,3189,3189,2807,2,3,0,1528942465499,1528948008587,1.0,1.5,1.298915,0.0
4,35628099,1,407,407,48,2,2,0,1528942466051,1528942466051,2.0,2.0,0.017398,0.0
